## Abstract
Despite the fact that the agricultural sector is a national economic development priority in sub-Saharan Africa, crop pests and diseases have been the challenge affecting major food security crops like maize. 
Maize Leaf Blight, also known as Northern Corn Leaf Blight has become a menace in low land agro-ecologies, during the last decade. On the other hand, according to research, Maize Streak Disease which is caused by the Maize Streak Virus is regarded as the third most serious disease affecting maize in sub-Saharan Africa. 
The prominence of these diseases has greatly affected the yields of Africa’s most important food crop. 

## Classes
<div style="display: flex; justify-content: center;">
    <div style="text-align: center; margin-right: 20px;">
        <p>Healthy</p>
        <img src="sample/1621590060253.jpg" alt="Image 1" style="width: 200;">
    </div>
    <div style="text-align: center;">
        <p>Maize Leaf Blight</p>
        <img src="sample/1621319276554.jpg" alt="Image 2" style="width: 200;">
    </div>
</div>


In [73]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import optimizers
import keras
import numpy as np
from glob import glob
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [74]:
# params
LEARNING_RATE = 0.02
CLASSES = 2
EPOCHS = 2
INCLUDE_TOP = False
BATCH_SIZE = 16
WEIGHTS = "imagenet"
IMAGE_SIZE = [224, 224, 3]
DECAY = 1e-6
MOMENTUM = 0.9

In [75]:
# data path
DATA = "data/"

In [76]:
# base model
base_model = VGG16(input_shape=IMAGE_SIZE, weights=WEIGHTS, include_top=INCLUDE_TOP)


In [77]:
# base model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [78]:
# freeze hidden layers
for layer in base_model.layers:
    layer.trainable = False

In [79]:
# create custom output(dense) layer
output = Dense(CLASSES, activation="softmax") (Flatten() (base_model.output))

In [80]:
# custom model
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(
            optimizer=optimizers.SGD(learning_rate=LEARNING_RATE, decay=DECAY, momentum=MOMENTUM),
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
new_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [81]:
# Augmentation
datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )
dataflow_kwargs = dict(
            target_size=IMAGE_SIZE[:-1],
            batch_size=BATCH_SIZE,
            interpolation="bilinear"
        )
train_datagenerator = ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=20,
                height_shift_range=20,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
validation_datagenerator = ImageDataGenerator(
               **datagenerator_kwargs
            )



In [82]:
# train test sets
train_set = train_datagenerator.flow_from_directory(
            directory=DATA,
            subset='training',
            shuffle=True,
            class_mode="categorical",
            **dataflow_kwargs
        )
validation_set = validation_datagenerator.flow_from_directory(
            directory=DATA,
            subset='validation',
            shuffle=True,
            class_mode="categorical",
            **dataflow_kwargs
        )

Found 960 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [83]:
# training
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

os.makedirs('artifacts/', exist_ok=True)

steps_per_epoch = train_set.samples // train_set.batch_size
validation_steps = validation_set.samples // validation_set.batch_size

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-5)

checkpoint = ModelCheckpoint(filepath='artifacts/model.h5',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint, lr_reducer]
start = datetime.now()

new_model.fit_generator(generator=train_set,
                        validation_data=validation_set,
                        epochs=EPOCHS,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps,
                        callbacks=callbacks,
                        verbose=1)

duration = datetime.now() - start
print(f"Training completed in time: {duration}")

/tmp/ipykernel_71642/3910870658.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  new_model.fit_generator(generator=train_set,


Epoch 1/2


2024-04-24 23:47:26.226118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - ETA: 0s - loss: 14.2223 - accuracy: 0.7500

2024-04-24 23:55:00.664281: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 3.52691, saving model to artifacts/model.h5
60/60 [==============================] - 566s 9s/step - loss: 14.2223 - accuracy: 0.7500 - val_loss: 3.5269 - val_accuracy: 0.9333 - lr: 0.0200
Epoch 2/2
60/60 [==============================] - ETA: 0s - loss: 5.6293 - accuracy: 0.8938
Epoch 2: val_loss did not improve from 3.52691
60/60 [==============================] - 608s 10s/step - loss: 5.6293 - accuracy: 0.8938 - val_loss: 7.4432 - val_accuracy: 0.8417 - lr: 0.0200
Training completed in time: 0:19:35.448157
